In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import time

from bs4 import BeautifulSoup

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

driver.get("https://itviec.com/sign_in")

driver.find_element(By.XPATH, "//input[@id='user_email']").send_keys("hieuvo8320@gmail.com")
driver.find_element(By.XPATH, "//input[@id='user_password']").send_keys("Aa@123456")
driver.find_element(By.XPATH, "//button[@type='submit']").click()

[WDM] - Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.61M/8.61M [00:10<00:00, 862kB/s]


In [2]:
def navigate_job_fresher():
    select_level = driver.find_element(By.XPATH, "//a[@class='btn btn-light dropdown-toggle']")
    driver.execute_script("arguments[0].click();", select_level)

#     select_level.click()
    fresher_level = driver.find_element(By.XPATH, "//input[@id='job_level_names_fresher']")
    driver.execute_script("arguments[0].click();", fresher_level)

    click_filter = driver.find_element(By.XPATH, "//button[@class='btn btn-primary text-white']")
    driver.execute_script("arguments[0].click();", click_filter)

In [3]:
def navigate_job_junior():
    select_level = driver.find_element(By.XPATH, "//a[@class='btn btn-light dropdown-toggle']")
    driver.execute_script("arguments[0].click();", select_level)

    # delete filter fresher
    delete_fresher = driver.find_element(By.XPATH, "//input[@id='job_level_names_fresher']")
    driver.execute_script("arguments[0].click();", delete_fresher)

    # choose filter junior
    junior_level = driver.find_element(By.XPATH, "//input[@id='job_level_names_junior']")
    driver.execute_script("arguments[0].click();", junior_level)

    click_filter = driver.find_element(By.XPATH, "//button[@class='btn btn-primary text-white']")
    driver.execute_script("arguments[0].click();", click_filter)

In [4]:
def navigate_job_senior():
    select_level = driver.find_element(By.XPATH, "//a[@class='btn btn-light dropdown-toggle']")
    driver.execute_script("arguments[0].click();", select_level)
    
     # delete filer junior
    delete_junior = driver.find_element(By.XPATH, "//input[@id='job_level_names_junior']")
    driver.execute_script("arguments[0].click();", delete_junior)

    # choose filter senior
    senior_level = driver.find_element(By.XPATH, "//input[@id='job_level_names_senior']")
    driver.execute_script("arguments[0].click();", senior_level)
    
    click_filter = driver.find_element(By.XPATH, "//button[@class='btn btn-primary text-white']")
    driver.execute_script("arguments[0].click();", click_filter)

In [5]:
def read_all_jd_by_level():
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # find all of items of this page
    
    ul = soup.find_all('ul', class_="pagination")
        
    items = 2
    
    # pagination page
    
    if len(ul) > 0:
        a = ul[0].find_all("a")
        items = int(a[-2].text)
    
    offer_and_skill_fresher = []

    for i in range(items - 1):
        job_java_fresher = soup.find_all('div', class_="job__description")

        for des in job_java_fresher:
            offer_and_skill = {}

            range_offer = des.find_all("div", class_="svg-icon__text")
            offer_and_skill["offer"] = range_offer[0].text

            skills = des.find_all("a", class_="job__skill ilabel mkt-track")
            offer_and_skill["skill"] = [skill.text.replace("\n", "") for skill in skills ]

            offer_and_skill_fresher.append(offer_and_skill)
            
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        
        time.sleep(2)
        
        # click net page if page has more than 2 pages
        if len(ul) > 0:
            time.sleep(2)
            next_page = driver.find_element(By.XPATH, "//a[@rel='next']")
            next_page.click()
        
        time.sleep(2)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

                                          
    return offer_and_skill_fresher

In [6]:
def find_all_jd(url):
    # navagite page via url
    driver.get(url)
    time.sleep(2)
    
    # init
    jds = []
    
    # navigate level fresher
    navigate_job_fresher()
    time.sleep(2)  # sleep for 2 seconds for fullfill page loaded
    
    jds_fresher =  read_all_jd_by_level()
    jds_fresher.append({"level": "fresher"})
    jds.append(jds_fresher)
    
    # navigate level junior
    navigate_job_junior()
    time.sleep(2)  # sleep for 5 seconds for fullfill page loaded
    
    jds_junior = read_all_jd_by_level()
    jds_junior.append({"level": "junior"})
    jds.append(jds_junior)
    
    # nagigate level senior
    navigate_job_senior()
    time.sleep(2)  # sleep for 5 seconds for fullfill page loaded
    
    jds_senior = read_all_jd_by_level()
    jds_senior.append({"level": "senior"})
    jds.append(jds_senior)
    
    return jds


In [180]:
java_job_url = "https://itviec.com/it-jobs/java?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=software-engineer-java-net-aureole-information-technology-ait-5620"
java_jds = find_all_jd(java_job_url)

In [216]:
nodejs_job_url = "https://itviec.com/it-jobs/nodejs?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=web-developer-ruby-on-rails-php-nodejs-motorist-pte-ltd-2941"
node_jds = find_all_jd(nodejs_job_url)

In [221]:
reactjs_job_url = "https://itviec.com/it-jobs/reactjs?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=mid-sr-reactjs-nextjs-developer-techvsi-3820"
react_jds = find_all_jd(reactjs_job_url)

In [223]:
ruby_on_rails_url = "https://itviec.com/it-jobs/ruby-on-rails?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=backend-developer-ruby-on-rails-mysql-motorist-pte-ltd-4357"
ruby_jds = find_all_jd(ruby_on_rails_url)

In [231]:
ios_url = "https://itviec.com/it-jobs/ios?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=ios-developer-objective-c-swift-up-to-1500-viettel-group-3209"
ios_jds = find_all_jd(ios_url)

In [236]:
android_url = "https://itviec.com/it-jobs/android?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=mobile-developer-android-ios-motorist-pte-ltd-3231"
android_jds = find_all_jd(android_url)

In [238]:
php_url = "https://itviec.com/it-jobs/php?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=senior-reactjs-developer-angular-aglie-vpbank-2832"
php_jds = find_all_jd(php_url)

In [240]:
python_url = "https://itviec.com/it-jobs/python?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=fullstack-developer-golang-python-reactjs-viettel-group-3810"
python_jds = find_all_jd(python_url)

In [7]:
c_url = "https://itviec.com/it-jobs/c++?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=embedded-software-engineer-terralogic-5830"
c_jds = find_all_jd(c_url)

In [245]:
golang_url = "https://itviec.com/it-jobs/golang?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=fullstack-developer-golang-python-reactjs-viettel-group-3810"
golang_jds = find_all_jd(golang_url)

In [247]:
angular_url = "https://itviec.com/it-jobs/angular?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=senior-developer-angular-javascript-typescript-athenapros-5317"
angular_jds = find_all_jd(angular_url)


In [249]:
vue_url = "https://itviec.com/it-jobs/vuejs?utm_campaign=gsn_brand_hcm&utm_medium=cpc&utm_source=google&utm_term=itviec&job_selected=lap-trinh-vien-reactjs-vuejs-java-co-the-di-lam-sau-tet-trung-tam-cong-nghe-thong-tin-mobifone-4744"
vue_jds = find_all_jd(vue_url)


In [252]:
vue_jds

[[{'offer': "You'll love it", 'skill': ['Java', 'ReactJS', 'VueJS']},
  {'offer': '500 - 1,200 USD',
   'skill': ['JavaScript', 'React Native', 'VueJS']},
  {'offer': '500 - 3,000 USD',
   'skill': ['PHP', 'Laravel', 'VueJS', 'Fresher Accepted']},
  {'offer': '500 - 1,500 USD', 'skill': ['Xamarin', 'C#', 'VueJS']},
  {'offer': '1,500 - 2,500 USD',
   'skill': ['PHP', 'Golang', 'VueJS', 'Fresher Accepted']},
  {'offer': "You'll love it",
   'skill': ['VueJS', 'AngularJS', 'ReactJS', 'Fresher Accepted']},
  {'offer': '1,000 - 1,500 USD',
   'skill': ['Python', 'Django', 'VueJS', 'Fresher Accepted']},
  {'level': 'fresher'}],
 [{'offer': "You'll love it", 'skill': ['Java', 'ReactJS', 'VueJS']},
  {'offer': '1,000 - 3,500 USD', 'skill': ['VueJS', 'JavaScript', 'ReactJS']},
  {'offer': "You'll love it", 'skill': ['ReactJS', 'JavaScript', 'VueJS']},
  {'offer': '1,500 - 4,000 USD', 'skill': ['NodeJS', 'ReactJS', 'VueJS']},
  {'offer': '500 - 1,200 USD',
   'skill': ['JavaScript', 'React Nati

In [154]:
import re

def proccessing(jds):
    result = []
    
    for i in range(len(jds) - 1):
        result.append({"salary": jds[i]["offer"]})
        
    return result

In [189]:
offer_fresher_java = proccessing(java_jds[0])
offer_junior_java = proccessing(java_jds[1])
offer_senior_java = proccessing(java_jds[2])

In [156]:
offer_fresher_nodejs = proccessing(node_jds[0])
offer_junior_nodejs = proccessing(node_jds[1])
offer_senior_nodejs = proccessing(node_jds[2])

In [157]:
offer_fresher_reactjs = proccessing(react_jds[0])
offer_junior_reactjs = proccessing(react_jds[1])
offer_senior_reactjs = proccessing(react_jds[2])

In [158]:
offer_fresher_ruby = proccessing(ruby_jds[0])
offer_junior_ruby = proccessing(ruby_jds[1])
offer_senior_ruby = proccessing(ruby_jds[2])

In [159]:
offer_fresher_android = proccessing(android_jds[0])
offer_junior_android = proccessing(android_jds[1])
offer_senior_android = proccessing(android_jds[2])

In [160]:
offer_fresher_php = proccessing(php_jds[0])
offer_junior_php = proccessing(php_jds[1])
offer_senior_php = proccessing(php_jds[2])

In [161]:
offer_fresher_ios = proccessing(ios_jds[0])
offer_junior_ios = proccessing(ios_jds[1])
offer_senior_ios = proccessing(ios_jds[2])

In [165]:
offer_senior_ios

[{'salary': 'Up to 1500$'},
 {'salary': "You'll love it"},
 {'salary': "You'll love it"},
 {'salary': '1,200 - 2,000 USD'},
 {'salary': 'Very competitive salary!'},
 {'salary': '1,000 - 2,000 USD'},
 {'salary': 'Up to $2000'},
 {'salary': 'Up to 3500$'},
 {'salary': "You'll love it"},
 {'salary': "You'll love it"},
 {'salary': '1,200 - 2,000 USD'},
 {'salary': "You'll love it"},
 {'salary': '2,000 - 4,000 USD'},
 {'salary': 'Negoti'},
 {'salary': "You'll love it"},
 {'salary': '500 - 1,800 USD'},
 {'salary': "You'll love it"},
 {'salary': "You'll love it"},
 {'salary': "You'll love it"},
 {'salary': "You'll love it"}]

In [21]:
def remove_duplicate(data):
    result = []
    for i in data:
        if i not in result:
            result.append(i)
            
    return result

In [172]:
import csv


header = ['java', 'nodejs', 'reactjs', 'ruby', 'android', 'ios', 'php', 'python', 'c++', 'golang', 'angular',
          'vue', 'level', 'salary']

with open('salary.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    
    writer.writerow(header)

    # write the java jds
    
    write_jds_to_csv(0, offer_fresher_java, "fresher", writer)
    write_jds_to_csv(0, offer_junior_java, "junior", writer)
    write_jds_to_csv(0, offer_senior_java, "senior", writer)
    
    #write the nodejs jds

    write_jds_to_csv(1, offer_fresher_nodejs, "fresher", writer)
    write_jds_to_csv(1, offer_junior_nodejs, "junior", writer)
    write_jds_to_csv(1, offer_senior_nodejs, "senior", writer)
    
    # reactjs
    
    write_jds_to_csv(2, offer_fresher_reactjs, "fresher", writer)
    write_jds_to_csv(2, offer_junior_reactjs, "junior", writer)
    write_jds_to_csv(2, offer_senior_reactjs, "senior", writer)

    # ruby
    
    write_jds_to_csv(3, offer_fresher_ruby, "fresher", writer)
    write_jds_to_csv(3, offer_junior_ruby, "junior", writer)
    write_jds_to_csv(3, offer_senior_ruby, "senior", writer)
    
    # android
    
    write_jds_to_csv(4, offer_fresher_android, "fresher", writer)
    write_jds_to_csv(4, offer_junior_android, "junior", writer)
    write_jds_to_csv(4, offer_senior_android, "senior", writer)
    
    # ios
    
    write_jds_to_csv(5, offer_fresher_ios, "fresher", writer)
    write_jds_to_csv(5, offer_junior_ios, "junior", writer)
    write_jds_to_csv(5, offer_senior_ios, "senior", writer)    
    
    # php
    
    write_jds_to_csv(6, offer_fresher_php, "fresher", writer)
    write_jds_to_csv(6, offer_junior_php, "junior", writer)
    write_jds_to_csv(6, offer_senior_php, "senior", writer)
    
    # pyhon
    
    write_jds_to_csv(7, proccessing(python_jds[0]), "fresher", writer)
    write_jds_to_csv(7, proccessing(python_jds[1]), "junior", writer)
    write_jds_to_csv(7, proccessing(python_jds[2]), "senior", writer)
    
    # c++
    write_jds_to_csv(8, proccessing(c_jds[0]), "fresher", writer)
    write_jds_to_csv(8, proccessing(c_jds[1]), "junior", writer)
    write_jds_to_csv(8, proccessing(c_jds[2]), "senior", writer)
    
    # golang
    write_jds_to_csv(9, proccessing(golang_jds[0]), "fresher", writer)
    write_jds_to_csv(9, proccessing(golang_jds[1]), "junior", writer)
    write_jds_to_csv(9, proccessing(golang_jds[2]), "senior", writer)
    
    # angular
    write_jds_to_csv(10, proccessing(angular_jds[0]), "fresher", writer)
    write_jds_to_csv(10, proccessing(angular_jds[1]), "junior", writer)
    write_jds_to_csv(10, proccessing(angular_jds[2]), "senior", writer)
    
    # vue
    
    write_jds_to_csv(11, proccessing(vue_jds[0]), "fresher", writer)
    write_jds_to_csv(11, proccessing(vue_jds[1]), "junior", writer)
    write_jds_to_csv(11, proccessing(vue_jds[2]), "senior", writer)
    

In [254]:
def write_jds_to_csv(job_jds, writer):
    list_skills = ['java', 'nodejs', 'reactjs', 'ruby', 'android', 'ios', 'php', 'python', 'c++', 'golang', 'angular']
    # 14 header in csv file
    
    for job_by_level in job_jds:
        job_by_level_len = len(job_by_level)
        
        for i in range(job_by_level_len - 1):
            salary_with_skills = [0] * 13

            # calculate offer 
            salary_with_skills[12] = job_by_level[i]["offer"]
            
            # calculate the skill
            
            for skill in job_by_level[i]["skill"]:
                skill_lower_case = skill.lower()
                if skill_lower_case in list_skills:
                    salary_with_skills[list_skills.index(skill_lower_case)] = 1
                    
            # calculate level
                salary_with_skills[11] = job_by_level[job_by_level_len - 1]["level"]
                
            writer.writerow(salary_with_skills)
        
            
            

In [256]:
import csv


header = ['java', 'nodejs', 'reactjs', 'ruby', 'android', 'ios', 'php', 'python', 'c++', 'golang', 'angular', 'level', 'salary']

with open('salary.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    
    writer.writerow(header)

    # write the java jds
    
    write_jds_to_csv(java_jds, writer)
    write_jds_to_csv(node_jds, writer)
    write_jds_to_csv(react_jds, writer)
    write_jds_to_csv(ruby_jds, writer)
    write_jds_to_csv(ios_jds, writer)
    write_jds_to_csv(android_jds, writer)
    write_jds_to_csv(php_jds, writer)
    write_jds_to_csv(python_jds, writer)
    write_jds_to_csv(c_jds, writer)
    write_jds_to_csv(golang_jds, writer)
    write_jds_to_csv(angular_jds, writer)
    